In [ ]:
import visa
import instruments as ik
import time
import numpy as np
from PyAPT import APTMotor

In [ ]:
import numpy as np
from numpy import pi, r_
import matplotlib.pyplot as plt
from scipy import optimize

In [ ]:
#We set up our power meter

rm = visa.ResourceManager()
p=rm.list_resources()
print(p)
powerMeter=ik.thorlabs.PM100USB.open_visa(p[0])
powerMeter.read()

In [ ]:
motorino=APTMotor(27250652)   #the argument is the motor ID

#we check what is the beginning position of the motor:
inPos=motorino.getPos()
print(inPos)

# and reset it to its zero position; note that there may be an offset between the motor 
# and your instrument, which in our case is an LC, so an optics that can browse between 0 and 
# 360 degrees:
motorino.go_home()

In [ ]:
# We perform the LC caracterization via the motor; in order to do that, we implement 
# a simple parabolic fit.

In [ ]:
alpha=0  #angle we suspect corresponds to the minimum
power=[]
for(i=alpha-10, i<alpha+10, i++):   #we keep the power meter value around the minimum
    power.append(powerMeter.read())

In [ ]:
fitfunc = lambda p, x: p[0]+p[1]*x+p[2]*x**2  # Target function
time = np.linspace(25, 45, 40)
errfunc = lambda p, x, y: fitfunc(p, x) - y # Distance to the target function
p0 = [100, 0, 1] # Initial guess for the parameters
p1, success = optimize.leastsq(errfunc, p0[:], args=(x, power))

alpha_min=np.argmin(fitfunc(p1,time))   #angle the fit calculated to be the minimum

plt.plot(x, power, "ro", time,fitfunc(p1, time),  "r-") # Plot of the data and the fit
plt.show()


In [ ]:
# We want to save pictures of the output mode via the CCD that is at the edn of our 
# experimental setup throughout the span of our LC between -45 degrees and 0 degrees. We
# capture a frame every second. We displace our LC of an angle dx. The time needed to 
#perform such a displacemnt is tr. We want to take 250 pictures. Hence we make a cycle 
#synchronized with these instrumental times.

In [ ]:
Nsteps=250
dx=0.18
tr=7.2*10**-3
t=1-tr
power=np.zeros(shape=Nsteps)
for i in range(Nsteps):
    print(motorino.getPos())
    motorino.mRel(dx)
    time.sleep(t)